# Notebook to evaluate *ritme* trials tracked with MLflow

This notebook shows how the output from the method `find_best_model_config` of the *ritme* package can be evaluated when the experiments are tracked with MLflow.

The only user input required is the path to the MLflow logs (variable `log_folder_location`). For reproducibility the exact experiment is rerun (section [Run experiment](#run-experiment)) if it does not exist yet (depending on your compute power this might take a while to run).


## Setup

In [ ]:
import warnings

import mlflow

from ritme.evaluate_mlflow import (
    barplot_metric,
    extract_run_config,
    parallel_coordinates_plot,
    plot_avg_history_per_model_type,
    plot_complexity_vs_metric,
    plot_metric_history_per_model_type,
    post_process_data_transform,
    violinplot_metric,
)

warnings.filterwarnings("ignore", category=FutureWarning)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
######## USER INPUTS ########

# path to MLflow logs
log_folder_location = "ritme_example_logs/trials_mlflow/mlruns"

# whether to display all trials (top_n=None) or just top N trials
top_n = None

# static or dynamic plots (static_plots=False) if available?
static_plots = True
######## END USER INPUTS #####

## Run experiment

In [ ]:
! ./run_experiment_mlflow.sh

## Extract trial information

In [ ]:
# extract all trial information
mlflow.set_tracking_uri(log_folder_location)

all_trials = mlflow.search_runs(
    order_by=["metrics.rmse_val ASC"], search_all_experiments=True
)

print(f"Found {all_trials.shape[0]} trials")

In [ ]:
# post-process data_transform entry - only needed in MLflow since WandB updates
# these automatically
all_trials = post_process_data_transform(all_trials)

## Insights on performance per model type

In [ ]:
violinplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col="params.model",
    group_name="Model type",
)

In [ ]:
barplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col="params.model",
    group_name="Model type",
    display_trial_name=True,  # if true display trial name on the x-axis
    top_n=top_n,  # to plot top N models, set to an integer value
)

## Model complexity vs. performance

In [ ]:
plot_complexity_vs_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col="params.model",
    group_name="Model type",
    static=static_plots,  # Set to False for interactive plotly graph
)

In [ ]:
violinplot_metric(
    all_trials,
    metric_col="metrics.nb_features",
    metric_name="Number of features",
    group_col="params.model",
    group_name="Model type",
)

In [ ]:
first_metric_col = "metrics.nb_features"
first_metric_name = "# features"
parallel_coordinates_plot(all_trials, first_metric_col, first_metric_name)

## Performance vs. feature engineering

### Effect of data transformation

In [ ]:
var = "params.data_transform"
var_name = "Data transform"

barplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
    display_trial_name=True,  # if true display trial name on the x-axis
    top_n=top_n,  # to plot top N models, set to an integer value
)

In [ ]:
violinplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
)

In [ ]:
parallel_coordinates_plot(all_trials, var, var_name, first_metric_cat=True)

### Effect of data selection

In [ ]:
var = "params.data_selection"
var_name = "Data selection"

barplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
    display_trial_name=True,  # if true display trial name on the x-axis
    top_n=top_n,  # to plot top N models, set to an integer value
)

In [ ]:
violinplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
)

In [ ]:
parallel_coordinates_plot(all_trials, var, var_name, first_metric_cat=True)

### Effect of data aggregation

In [ ]:
var = "params.data_aggregation"
var_name = "Data aggregation"

barplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
    display_trial_name=True,  # if true display trial name on the x-axis
    top_n=top_n,  # to plot top N models, set to an integer value
)

In [ ]:
violinplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
)

In [ ]:
parallel_coordinates_plot(all_trials, var, var_name, first_metric_cat=True)

### Effect of data enrichment

In [ ]:
var = "params.data_enrich"
var_name = "Data enrichment"

barplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
    display_trial_name=True,  # if true display trial name on the x-axis
    top_n=top_n,  # to plot top N models, set to an integer value
)

In [ ]:
violinplot_metric(
    all_trials,
    metric_col="metrics.rmse_val",
    metric_name="RMSE Validation",
    group_col=var,
    group_name=var_name,
)

In [ ]:
parallel_coordinates_plot(all_trials, var, var_name, first_metric_cat=True)

## Training over time

In [ ]:
metric = "rmse_train"
client = mlflow.tracking.MlflowClient(tracking_uri=log_folder_location)

# one figure per model type
for model_type in all_trials["params.model"].unique():
    model_trials = all_trials[all_trials["params.model"] == model_type]
    plot_metric_history_per_model_type(metric, client, model_trials)

In [ ]:
# plot all metric history in one plot
plot_metric_history_per_model_type(metric, client, all_trials)

* If per model type later launched trials yield smaller RMSE train then the selected search algorithm works
* *Note:* The classification network (`nn_class`) is trained by minimizing the cross‐entropy loss, and the ordinal‐regression variant (`nn_corn`) is trained by minimizing the CORN loss.  herefore, training curves based on MSE‐derived metrics (e.g. RMSE) are not guaranteed to decrease monotonically during optimization for these models.

In [ ]:
plot_avg_history_per_model_type("rmse_train", client, all_trials)

In [ ]:
plot_avg_history_per_model_type("rmse_val", client, all_trials)

## Run configuration overview

In [ ]:
run_config = extract_run_config(all_trials)
run_config